# **Outliers analysis**

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('games_with_relationship.csv')
data

,ID,Date,Consent,Team_Name,Selected_Language,Consent_Time_X_axis,Age_X_axis,Companionship_X_axis,Question_Time_X_axis,Consent_Time_Y_axis,...,Track_Time,Mistake,Track_Completed,Interface_Mode,GAME_ID,Age_Group_X,Age_Group_Y,Relationship_X,Relationship_Y,Relationship_Name
0,20240102-0000,20240102,True,BJ2 00,0,09:30:48:500,25.0,1,09:30:55:925,09:30:27:247,...,44594,False,True,[],20240102-0000_BJ200_2_09-31-04-583,18-29,18-29,Znajomy,Znajomy,Znajomy - Znajomy
1,20240102-0000,20240102,True,BJ2 00,0,09:30:48:500,25.0,1,09:30:55:925,09:30:27:247,...,30630,True,False,[],20240102-0000_BJ200_4_09-32-10-982,18-29,18-29,Znajomy,Znajomy,Znajomy - Znajomy
2,20240102-0000,20240102,True,BJ2 00,0,09:30:48:500,25.0,1,09:30:55:925,09:30:27:247,...,1267,True,False,[],20240102-0000_BJ200_4_09-32-48-653,18-29,18-29,Znajomy,Znajomy,Znajomy - Znajomy
3,20240102-0000,20240102,True,BJ2 00,0,09:30:48:500,25.0,1,09:30:55:925,09:30:27:247,...,14147,False,True,[],20240102-0000_BJ200_4_09-32-58-146,18-29,18-29,Znajomy,Znajomy,Znajomy - Znajomy
4,20240102-0000,20240102,True,BJ2 00,0,09:30:48:500,25.0,1,09:30:55:925,09:30:27:247,...,5038,True,False,[],20240102-0000_BJ200_6_09-33-24-189,18-29,18-29,Znajomy,Znajomy,Znajomy - Znajomy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128174,20241231-0049,20241231,True,BDV 49,0,13:24:58:149,5.0,0,13:25:09:895,13:24:58:250,...,4103,True,False,[],20241231-0049_BDV49_13_13-35-38-923,< 18,< 18,Członek rodziny,Członek rodziny,Członek rodziny - członek rodziny
128175,20241231-0049,20241231,True,BDV 49,0,13:24:58:149,5.0,0,13:25:09:895,13:24:58:250,...,205671,False,True,[],20241231-0049_BDV49_13_13-35-48-915,< 18,< 18,Członek rodziny,Członek rodziny,Członek rodziny - członek rodziny
128176,20241231-0049,20241231,True,BDV 49,0,13:24:58:149,5.0,0,13:25:09:895,13:24:58:250,...,69152,False,True,[],20241231-0049_BDV49_12_13-39-33-306,< 18,< 18,Członek rodziny,Członek rodziny,Członek rodziny - członek rodziny
128177,20241231-0049,20241231,True,BDV 49,0,13:24:58:149,5.0,0,13:25:09:895,13:24:58:250,...,87220,True,False,[],20241231-0049_BDV49_13_13-40-50-734,< 18,< 18,Członek rodziny,Członek rodziny,Członek rodziny - członek rodziny


In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Funkcja do wykrywania outlierów metodą Z-Score
def detect_outliers_zscore(series, threshold=3):
    mean = series.mean()
    std = series.std()
    z_scores = (series - mean) / std
    return np.abs(z_scores) > threshold, z_scores

# Funkcja do czyszczenia nielegalnych znaków w nazwach plików
def sanitize_filename(filename):
    # Lista nielegalnych znaków w nazwach plików w systemie Windows
    illegal_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in illegal_chars:
        filename = filename.replace(char, '_')
    filename = filename.replace('<', 'lt')  # Zastąpienie '<' słowem 'lt'
    return filename

# Ścieżka główna do zapisu wykresów
base_dir = "wykresy_outliery"

# Utwórz główny folder, jeśli nie istnieje
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Analiza
for track_id in sorted(data['Track_ID'].unique()):
    track_data = data[data['Track_ID'] == track_id]
    
    print(f"\n--- Analiza dla toru {track_id} ---")

    # Tworzymy folder dla danego toru
    track_dir = os.path.join(base_dir, f"tor_{track_id}")
    if not os.path.exists(track_dir):
        os.makedirs(track_dir)
    
    # --- 1. Relationship_Name ---
    rel_outliers = []  # Lista do przechowywania wyników dla relacji
    for relationship in sorted(track_data['Relationship_Name'].dropna().unique()):
        group_data = track_data[track_data['Relationship_Name'] == relationship].copy()
        
        if group_data.empty:
            continue

        outliers, z_scores = detect_outliers_zscore(group_data['Track_Time'])
        group_data['Outlier'] = outliers
        group_data['Z_Score'] = z_scores

        print(f"\nRelationship: {relationship}")
        print(f"Liczba outlierów: {outliers.sum()}")

        # Tworzymy folder dla relacji, jeśli nie istnieje
        rel_dir = os.path.join(track_dir, 'relacja')
        if not os.path.exists(rel_dir):
            os.makedirs(rel_dir)

        # Wykres Track_Time vs index
        plt.figure(figsize=(14,6))
        plt.subplot(1,2,1)
        sns.scatterplot(x=group_data.index, y=group_data['Track_Time'], hue=group_data['Outlier'],
                        palette={True: 'red', False: 'blue'}, s=50)
        plt.title(f'Track_Time vs Index - Track_ID {track_id} | {relationship}')
        plt.axhline(group_data['Track_Time'].mean(), color='green', linestyle='--', label='Mean')
        plt.legend(title='Outlier')
        plt.xlabel('Index')
        plt.ylabel('Track_Time (ms)')

        # Wykres Z-score
        plt.subplot(1,2,2)
        sns.scatterplot(x=group_data.index, y=group_data['Z_Score'], hue=group_data['Outlier'],
                        palette={True: 'red', False: 'blue'}, s=50)
        plt.axhline(3, color='orange', linestyle='--')
        plt.axhline(-3, color='orange', linestyle='--')
        plt.title(f'Z-Score vs Index - Track_ID {track_id} | {relationship}')
        plt.xlabel('Index')
        plt.ylabel('Z-Score')

        plt.tight_layout()
        
        # Zapisz wykresy do folderu
        plot_filename = os.path.join(rel_dir, f'tor_{track_id}_relacja_{sanitize_filename(str(relationship))}_outliers.png')
        plt.savefig(plot_filename)
        plt.close()

        # Zapisujemy dane do analizy później
        rel_outliers.append({'Relationship_Name': relationship, 'Outlier_Count': outliers.sum()})
    
    # --- 2. Age Groups ---
    age_x = track_data[['Age_Group_X', 'Track_Time']].rename(columns={'Age_Group_X': 'Age_Group'})
    age_y = track_data[['Age_Group_Y', 'Track_Time']].rename(columns={'Age_Group_Y': 'Age_Group'})
    age_combined = pd.concat([age_x, age_y], axis=0).dropna()
    
    age_outliers = []  # Lista do przechowywania wyników dla grup wiekowych
    for age_group in sorted(age_combined['Age_Group'].unique()):
        group_data = age_combined[age_combined['Age_Group'] == age_group].copy()
        
        if group_data.empty:
            continue

        outliers, z_scores = detect_outliers_zscore(group_data['Track_Time'])
        group_data['Outlier'] = outliers
        group_data['Z_Score'] = z_scores

        print(f"\nAge Group: {age_group}")
        print(f"Liczba outlierów: {outliers.sum()}")

        # Tworzymy folder dla grupy wiekowej, jeśli nie istnieje
        age_dir = os.path.join(track_dir, 'wiek')
        if not os.path.exists(age_dir):
            os.makedirs(age_dir)

        # Wykres Track_Time vs index
        plt.figure(figsize=(14,6))
        plt.subplot(1,2,1)
        sns.scatterplot(x=group_data.index, y=group_data['Track_Time'], hue=group_data['Outlier'],
                        palette={True: 'red', False: 'blue'}, s=50)
        plt.title(f'Track_Time vs Index - Track_ID {track_id} | Age Group {age_group}')
        plt.axhline(group_data['Track_Time'].mean(), color='green', linestyle='--', label='Mean')
        plt.legend(title='Outlier')
        plt.xlabel('Index')
        plt.ylabel('Track_Time (ms)')

        # Wykres Z-score
        plt.subplot(1,2,2)
        sns.scatterplot(x=group_data.index, y=group_data['Z_Score'], hue=group_data['Outlier'],
                        palette={True: 'red', False: 'blue'}, s=50)
        plt.axhline(3, color='orange', linestyle='--')
        plt.axhline(-3, color='orange', linestyle='--')
        plt.title(f'Z-Score vs Index - Track_ID {track_id} | Age Group {age_group}')
        plt.xlabel('Index')
        plt.ylabel('Z-Score')

        plt.tight_layout()
        
        # Zapisz wykresy do folderu
        plot_filename = os.path.join(age_dir, f'tor_{track_id}_wiek_{sanitize_filename(str(age_group))}_outliers.png')
        plt.savefig(plot_filename)
        plt.close()

        # Zapisujemy dane do analizy później
        age_outliers.append({'Age_Group': age_group, 'Outlier_Count': outliers.sum()})

    # Tworzymy ramki danych z wynikami
    rel_df = pd.DataFrame(rel_outliers)
    age_df = pd.DataFrame(age_outliers)
        
    # --- 3. Barplot charts - Number of Outliers for each Relationship and Age Group ---
    plt.figure(figsize=(10,6))
    sns.barplot(data=rel_df.sort_values('Outlier_Count', ascending=False),
                y='Relationship_Name', x='Outlier_Count', orient='h')  # Horizontal bars
    plt.title(f'Number of Outliers by Relationship - Track_ID {track_id}')
    plt.xlabel('Number of Outliers')
    plt.ylabel('Relationship')
    plt.tight_layout()

    # Save barplot chart for relationships
    rel_bar_filename = os.path.join(track_dir, 'relacja', f'tor_{track_id}_outliers_relationship.png')
    plt.savefig(rel_bar_filename)
    plt.close()

    plt.figure(figsize=(10,6))
    sns.barplot(data=age_df.sort_values('Outlier_Count', ascending=False),
                y='Age_Group', x='Outlier_Count', orient='h')  # Horizontal bars
    plt.title(f'Number of Outliers by Age Group - Track_ID {track_id}')
    plt.xlabel('Number of Outliers')
    plt.ylabel('Age Group')
    plt.tight_layout()

    # Zapisz wykres barplot dla grup wiekowych
    age_bar_filename = os.path.join(track_dir, 'wiek', f'tor_{track_id}_outliers_age_group.png')
    plt.savefig(age_bar_filename)
    plt.close()



--- Analiza dla toru 1 ---

Relationship: Członek rodziny - członek rodziny
Liczba outlierów: 140

Relationship: Inna znacząca - Inna znacząca
Liczba outlierów: 34

Relationship: Nieznajomy - Nieznajomy
Liczba outlierów: 21

Relationship: To skomplikowane
Liczba outlierów: 69

Relationship: Znajomy - Znajomy
Liczba outlierów: 73

Age Group: 18-29
Liczba outlierów: 118

Age Group: 30-44
Liczba outlierów: 122

Age Group: 45-59
Liczba outlierów: 34

Age Group: 60+
Liczba outlierów: 59

Age Group: < 18
Liczba outlierów: 350

--- Analiza dla toru 2 ---

Relationship: Członek rodziny - członek rodziny
Liczba outlierów: 157

Relationship: Inna znacząca - Inna znacząca
Liczba outlierów: 32

Relationship: Nieznajomy - Nieznajomy
Liczba outlierów: 21

Relationship: To skomplikowane
Liczba outlierów: 79

Relationship: Znajomy - Znajomy
Liczba outlierów: 98

Age Group: 18-29
Liczba outlierów: 161

Age Group: 30-44
Liczba outlierów: 114

Age Group: 45-59
Liczba outlierów: 45

Age Group: 60+
Liczba